In [ ]:
# !pip install rlcard[torch]

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [109]:
import rlcard
from rlcard.agents import RandomAgent
from rlcard.games.base import Card
import torch
from IPython.display import clear_output
from rlcard.utils import reorganize, print_card


In [4]:
env = rlcard.make(
    'limit-holdem',
    config={
        'seed': 42,
        'game_num_players': 3
    }
)
# agent1, agent2, agent3 = RandomAgent(num_actions=4), RandomAgent(num_actions=4), RandomAgent(num_actions=4)
agent1 = torch.load('/content/drive/MyDrive/CS6284/nfsp_comp1.pth')
agent2 = torch.load('/content/drive/MyDrive/CS6284/nfsp_comp2.pth')
agent3 = torch.load('/content/drive/MyDrive/CS6284/nfsp_comp3.pth')
env.set_agents([agent1, agent2, agent3])
agents = env.agents



In [124]:
game = env.game.init_game()

old_hands = [env.game.players[i].hand for i in range(env.num_players)]

cards = ['C7', 'H2', 'SA', 'HA', 'D8', 'S2']

env.game.players[0].hand = [Card('C', '7'), Card('H', '2')]
env.game.players[1].hand = [Card('S', 'A'), Card('H', 'A')]
env.game.players[2].hand = [Card('D', '8'), Card('S', '2')]

for h in old_hands:
  if str(h[0]) not in cards:
    env.game.dealer.deck.append(h[0])
  if str(h[0]) not in cards:
    env.game.dealer.deck.append(h[1])

trajectories = [[] for _ in range(env.num_players)]

curr_player_id = env.get_player_id()
curr_state = env.get_state(curr_player_id)

trajectories[curr_player_id].append(curr_state)

while not env.is_over():
  curr_player = agents[curr_player_id]
  curr_action = curr_player.eval_step(curr_state)
  
  trajectories[curr_player_id].append(curr_action)

  curr_state, curr_player_id = env.step(curr_action[0], curr_player.use_raw)


  if not env.game.is_over():
    trajectories[curr_player_id].append(curr_state)

for player_id in range(env.num_players):
  state = env.get_state(player_id)
  trajectories[player_id].append(state)


# Payoffs
payoffs = env.get_payoffs()

trajectories = reorganize(trajectories, payoffs)

In [125]:
def visualize_trajectory(traj, traj_env, player_id):
  p_traj = traj[player_id]

  round2name = {0: 'Pre Flop', 1: 'Flop', 2: 'Turn', 3: 'River'}
  all_actions = ['call', 'raise', 'fold', 'check']
  round_count = 0
  pub_cards = []

  print(f"Player {player_id} cards")
  print_card(p_traj[0][0]['raw_obs']['hand'])
  print("-"*50)
  print(round2name[round_count])
  print("-"*50)

  for t in p_traj:
    s, a, r, ns, done = t
    old_len = len(pub_cards)

    pub_cards = s['raw_obs']['public_cards']

    if old_len != len(pub_cards):
      round_count += 1
      print("-"*50)
      print(f"{round2name[round_count]}")
      print("-"*50)
      print_card(pub_cards) 

    print(all_actions[a[0]])

    if done:
      print("-"*50)
      print(f"Player {player_id} got {r}")



In [128]:
visualize_trajectory(trajectories, env, 2)

Player 2 cards
┌─────────┐   ┌─────────┐
│8        │   │2        │
│         │   │         │
│         │   │         │
│    ♦    │   │    ♠    │
│         │   │         │
│         │   │         │
│        8│   │        2│
└─────────┘   └─────────┘
--------------------------------------------------
Pre Flop
--------------------------------------------------
call
--------------------------------------------------
Flop
--------------------------------------------------
┌─────────┐   ┌─────────┐   ┌─────────┐
│10       │   │3        │   │5        │
│         │   │         │   │         │
│         │   │         │   │         │
│    ♥    │   │    ♣    │   │    ♠    │
│         │   │         │   │         │
│         │   │         │   │         │
│       01│   │        3│   │        5│
└─────────┘   └─────────┘   └─────────┘
raise
raise
fold
--------------------------------------------------
Player 2 got -6.0
